# Classification model using Multilayer Perceptron

### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

### 2. Upload Clinic data

In [2]:
path ='../../../Data_preprocessing/Prediction PFS/Clinical_data_categorized_PFS.csv' 
data = pd.read_csv(path)
data.head()

,Unnamed: 0,RNA_ID,Age,Sarc,Rhab,Number_of_Prior_Therapies,Days_from_TumorSample_Collection_and_Start_of_Trial_Therapy,Tumor_Shrinkage,PFS,TM_TC_Ratio,...,MSKCC_NA,MSKCC_POOR,IMDC_FAVORABLE,IMDC_INTERMEDIATE,IMDC_NOT_REPORTED,IMDC_POOR,ImmunoPhenotype_Desert,ImmunoPhenotype_Excluded,ImmunoPhenotype_Infiltrated,ImmunoPhenotype_NA
0,0,NaN,73,0.0,0.0,2.0,808,0.0,1.545205,-1.0,...,0,0,0,0,1,0,0,0,1,0
1,1,NaN,55,0.0,0.0,2.0,1826,0.0,3.419178,-1.0,...,0,0,0,0,1,0,1,0,0,0
2,2,NaN,51,0.0,0.0,1.0,1541,0.0,1.413699,-1.0,...,0,0,0,0,1,0,0,0,1,0
3,3,NaN,70,0.0,0.0,3.0,-1,0.0,0.394521,-1.0,...,0,1,0,0,1,0,0,0,0,1
4,4,NaN,59,0.0,0.0,3.0,376,0.0,10.158904,-1.0,...,0,1,0,0,1,0,0,0,1,0


In [3]:
Y = [] # Target column
# For each entry I classified it by its PFS value.
for i in range (len(data)):
    if data.PFS[i]<3: # If PFS is lower than 3 months, I will consider it as NonResponder (NR)
        Y.append(0)
    elif data.PFS[i]<6: # If PFS is over 6 months, I will consider it as Responder (R)
        Y.append(1)
    else:
        Y.append(2) # If PFS is between 3 and 6 months, I will consider it as SemiResponder (SR)
        
data = data.drop('PFS', axis=1) # As we won't need this column any more, I deleted it.

X = data.iloc[:,2:26] 
# I selected all the columns by removing the Unnamed column (row id) and the Target column.
print(X)
print('Number of patients: ',len(Y))

     Age  Sarc  Rhab  Number_of_Prior_Therapies  \
0     73   0.0   0.0                        2.0   
1     55   0.0   0.0                        2.0   
2     51   0.0   0.0                        1.0   
3     70   0.0   0.0                        3.0   
4     59   0.0   0.0                        3.0   
..   ...   ...   ...                        ...   
285   54   0.0   0.0                        3.0   
286   56   0.0   0.0                        2.0   
287   65   0.0   0.0                        2.0   
288   48   0.0   0.0                        1.0   
289   76   0.0   0.0                        2.0   

     Days_from_TumorSample_Collection_and_Start_of_Trial_Therapy  \
0                                                  808             
1                                                 1826             
2                                                 1541             
3                                                   -1             
4                                              

### 3. Train-Test dataset split

In [4]:
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.20, random_state=125)

print('Training set size:', len(XTrain))
print('Target column size of the training set:', len(yTrain))
print('Test set size:', len(XTest))
print('Target column size of the test set:', len(yTest))

Training set size: 232
Target column size of the training set: 232
Test set size: 58
Target column size of the test set: 58


### 4. Select the parameters of the model and fit it

In [5]:
param_grid = {'max_iter': [100000],
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'random_state': [125],
            'max_fun': [300, 500,1000, 5000, 10000, 15000, 20000],
            'hidden_layer_sizes': [3,5]}

# I created a GridSearchCV which allows us to systematically evaluate and select the parameters of our model.
# By indicating a model and the parameters to test, you can evaluate the performance of the first one based on the
# seconds through cross validation.
clf = GridSearchCV(MLPClassifier(), param_grid, cv =5)

clf.fit(XTrain , yTrain)


GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [3, 5],
                         'max_fun': [300, 500, 1000, 5000, 10000, 15000, 20000],
                         'max_iter': [100000], 'random_state': [125]})

In [6]:
print("Best estimate of parameters according to GridSearchCV:")
print(clf.best_estimator_)
# Fit the model with the best parameters
model = clf.best_estimator_

Best estimate of parameters according to GridSearchCV:
MLPClassifier(activation='logistic', hidden_layer_sizes=3, max_fun=300,
              max_iter=100000, random_state=125)


In [7]:
print("Best result of the cross validation of the model with the best paramters:" +str(clf.best_score_))

Best result of the cross validation of the model with the best paramters:0.45263644773358


### 5. Prediction

In [8]:
# Making predictions with the optimal model on the training dataset
yhatTrain = model.predict(XTrain)
contTrain = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(yTrain),1) :
    if (yhatTrain[i] == yTrain[i]):
        contTrain = contTrain + 1

In [9]:
# Making predictions with the optimal model on the test dataset
yhatTest = model.predict(XTest)
contTest = 0

# Comparing with the Target column and check how many hits there have been
for i in range(0,len(yTest),1) :
    if (yhatTest[i] == yTest[i]):
        contTest = contTest + 1


### 6. Results

In [10]:
print('Final accuracy on the training dataset:' + str(contTrain/len(yTrain)))
print('Final accuracy on the testing dataset: ' + str(contTest/len(yTest)))

Final accuracy on the training dataset:0.4525862068965517
Final accuracy on the testing dataset: 0.43103448275862066


In [11]:
from sklearn.metrics import classification_report,confusion_matrix

print('----------------Confusion Matrix (Training)------------------')
print(confusion_matrix(yTrain,yhatTrain))
print('Input data:  ' + str(np.array(yTrain)))
print('Prediction:        ' +str(yhatTrain))

----------------Confusion Matrix (Training)------------------
[[105   0   0]
 [ 54   0   0]
 [ 73   0   0]]
Input data:  [2 0 1 2 0 2 1 2 2 1 1 0 1 0 2 2 2 2 1 2 1 2 1 0 0 2 1 1 0 0 0 2 0 0 1 2 2
 2 0 0 2 0 2 2 0 2 1 2 0 0 2 0 2 0 0 0 0 0 1 0 2 1 2 0 2 0 2 2 2 2 2 2 0 2
 1 2 0 0 0 0 2 0 0 2 2 2 0 0 1 1 1 2 2 1 1 2 0 0 0 0 2 2 1 0 0 0 1 1 2 2 2
 2 0 0 2 0 0 0 0 2 0 1 2 1 2 2 0 1 0 0 0 0 0 0 0 1 0 2 2 1 0 0 0 0 2 2 2 0
 1 1 0 2 0 2 2 2 0 0 0 0 1 1 2 0 2 0 0 1 0 0 0 1 2 0 2 1 2 0 2 0 0 0 1 2 1
 1 0 1 0 1 1 1 0 0 2 1 0 2 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 2 1 1
 0 2 0 0 2 0 1 1 0 0]
Prediction:        [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 

In [12]:
print(classification_report(yTrain,yhatTrain))

              precision    recall  f1-score   support

           0       0.45      1.00      0.62       105
           1       0.00      0.00      0.00        54
           2       0.00      0.00      0.00        73

    accuracy                           0.45       232
   macro avg       0.15      0.33      0.21       232
weighted avg       0.20      0.45      0.28       232



C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
print('----------------Confusion Matrix (Test)------------------')
print(confusion_matrix(yTest,yhatTest))
print('Input data:  ' + str(np.array(yTest)))
print('Prediction:        ' +str(yhatTest))

----------------Confusion Matrix (Test)------------------
[[25  0  0]
 [15  0  0]
 [18  0  0]]
Input data:  [2 2 1 2 0 1 1 2 2 2 0 0 2 0 1 0 0 2 0 2 2 0 2 2 1 0 0 1 1 0 0 2 1 0 0 0 1
 1 1 1 2 2 0 1 0 0 0 0 1 0 2 0 2 1 0 2 0 0]
Prediction:        [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [14]:
print(classification_report(yTest,yhatTest))

              precision    recall  f1-score   support

           0       0.43      1.00      0.60        25
           1       0.00      0.00      0.00        15
           2       0.00      0.00      0.00        18

    accuracy                           0.43        58
   macro avg       0.14      0.33      0.20        58
weighted avg       0.19      0.43      0.26        58



C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sandr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
